In [ ]:
!pip install wandb

In [106]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
import wandb
import numpy as np
from wandb.keras import WandbCallback
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm

In [107]:
tf.__version__

'2.6.0'

In [ ]:
wandb.init(project='Face_Recognition')

In [109]:
folder_address = '/content/drive/MyDrive/Datasets/7-7 dataset/'

batch_size = 16
width = height = 224

config = wandb.config
config.learning_rate = 0.001

In [110]:
data_generator = ImageDataGenerator(
    rescale = 1. / 255,
    validation_split = 0.1,
    horizontal_flip = True
)

train_data = data_generator.flow_from_directory(folder_address,
                                                    target_size = (width, height),
                                                    class_mode = 'categorical',
                                                    batch_size = batch_size,
                                                    shuffle = True,
                                                    subset ='training')

val_data = data_generator.flow_from_directory(folder_address,
                                                    target_size = (width, height),
                                                    class_mode = 'categorical',
                                                    batch_size = batch_size,
                                                    shuffle = False,
                                                    subset ='validation')

Found 1228 images belonging to 14 classes.
Found 131 images belonging to 14 classes.


In [111]:
train_data.image_shape

(224, 224, 3)

In [113]:
class MyModel(tf.keras.Model):
  def __init__(self, number_of_classes):
    super().__init__()
    self.conv1 = Conv2D(32, (3, 3), activation='relu', input_shape = (width, height, 3))
    # self.conv2 = Conv2D(32, (3, 3), activation='relu')
    self.flatten = Flatten()
    self.fc1 = Dense(128, activation='relu')
    self.fc2 = Dense(64, activation='relu')
    self.fc3 = Dense(number_of_classes, activation='sigmoid')

  def call(self, x):

    x = self.conv1(x)
    # x = self.conv2(x)
    x = self.flatten(x)
    x = self.fc1(x)
    x = self.fc2(x)
    output = self.fc3(x)

    return output

model = MyModel(14)

In [114]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer =  tf.keras.optimizers.Adam()

In [115]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_acc = tf.keras.metrics.CategoricalAccuracy(name='train_acc')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_acc = tf.keras.metrics.CategoricalAccuracy(name='test_acc')

In [ ]:
configs = {
              "learning_rate": 0.001,
              "epochs": 20,
              "batch_size": 16,
              "log_step": 200,
              "val_log_step": 50
           }

run = wandb.init(project='Face_Recognition', config=configs)
config = wandb.config

In [117]:
def train_step(images, y):
    with tf.GradientTape() as tape:
        y_pred = model(images, training = True)
        loss = tf.keras.losses.categorical_crossentropy(y, y_pred)

    #gradient calculate
    gradients = tape.gradient(loss, model.trainable_variables)

    #update
    optimizer.apply_gradients(zip(gradients,model.trainable_variables))

    train_loss(loss)
    train_acc(y, y_pred)


In [118]:
def test_step(images, y):
  y_pred = model(images, training = False)
  loss = tf.keras.losses.categorical_crossentropy(y, y_pred)

  test_loss(loss)
  test_acc(y, y_pred)

In [119]:
def train():
    epochs = 20

    for epoch in range(epochs):

        train_loss.reset_state()
        train_acc.reset_state()
        test_loss.reset_state()
        test_acc.reset_state()

        count_for_train = 0
        count_for_test = 0

        i_train = len(train_data)
        i_test = len(val_data)

        for images, labels in tqdm(train_data):
            train_step(images, labels)
            count_for_train += 1
            if count_for_train > i_train:
                break

        for images, labels in tqdm(val_data):
            test_step(images, labels)
            count_for_test += 1
            if count_for_test > i_test:
                break

        print('epoch:', epoch + 1)
        print('loss:', train_loss.result())
        print('accuracy:', train_acc.result())
        print('val loss:', test_loss.result())
        print('val accuracy:', test_acc.result())

        wandb.log({'epochs': epoch,
            'loss': np.mean(train_loss.result()),
            'acc': float(train_acc.result()), 
            'test_loss': np.mean(test_loss.result()),
            'test_acc': float(test_acc.result())
            })

In [120]:
print(train_data)

In [121]:
train()

100%|██████████| 9/9 [00:02<00:00,  3.45it/s]


epoch: 1
loss: tf.Tensor(8.530527, shape=(), dtype=float32)
accuracy: tf.Tensor(0.19212219, shape=(), dtype=float32)
val loss: tf.Tensor(2.3961103, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.3469388, shape=(), dtype=float32)


100%|██████████| 9/9 [00:02<00:00,  3.40it/s]


epoch: 2
loss: tf.Tensor(1.5601741, shape=(), dtype=float32)
accuracy: tf.Tensor(0.5297428, shape=(), dtype=float32)
val loss: tf.Tensor(2.1384563, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.49659863, shape=(), dtype=float32)


100%|██████████| 9/9 [00:02<00:00,  3.43it/s]


...

epoch: 18
loss: tf.Tensor(0.00029818114, shape=(), dtype=float32)
accuracy: tf.Tensor(0.9991961, shape=(), dtype=float32)
val loss: tf.Tensor(0.7671376, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.8134328, shape=(), dtype=float32)


100%|██████████| 9/9 [00:02<00:00,  3.52it/s]


epoch: 19
loss: tf.Tensor(0.00025207343, shape=(), dtype=float32)
accuracy: tf.Tensor(0.9983923, shape=(), dtype=float32)
val loss: tf.Tensor(0.81086195, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.82312924, shape=(), dtype=float32)


100%|██████████| 9/9 [00:02<00:00,  3.45it/s]

epoch: 20
loss: tf.Tensor(0.00022466025, shape=(), dtype=float32)
accuracy: tf.Tensor(0.9991961, shape=(), dtype=float32)
val loss: tf.Tensor(0.8747432, shape=(), dtype=float32)
val accuracy: tf.Tensor(0.7823129, shape=(), dtype=float32)
